In [5]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load and reverse all dataframes
mass_balance_df = pd.read_csv('project-glaciers/data/mass_balance_hy.csv').iloc[::-1].reset_index(drop=True)
davos_dev_temp = pd.read_csv('project-glaciers/data/weather_dev6190_davos_temp.csv').iloc[::-1].reset_index(drop=True)
davos_dev_prec = pd.read_csv('project-glaciers/data/weather_dev6190_davos_prec.csv').iloc[::-1].reset_index(drop=True)
sion_dev_temp = pd.read_csv('project-glaciers/data/weather_dev6190_sion_temp.csv').iloc[::-1].reset_index(drop=True)
sion_dev_prec = pd.read_csv('project-glaciers/data/weather_dev6190_sion_prec.csv').iloc[::-1].reset_index(drop=True)
altdorf_dev_temp = pd.read_csv('project-glaciers/data/weather_dev6190_altdorf_temp.csv').iloc[::-1].reset_index(drop=True)
altdorf_dev_prec = pd.read_csv('project-glaciers/data/weather_dev6190_altdorf_prec.csv').iloc[::-1].reset_index(drop=True)

# Define glacier-weather station mappings
glacier_mappings = {
    'Grosser Aletschgletscher': {'temp': sion_dev_temp, 'prec': sion_dev_prec},
    'Allalingletscher': {'temp': sion_dev_temp, 'prec': sion_dev_prec},
    'Griesgletscher': {'temp': sion_dev_temp, 'prec': sion_dev_prec},
    'Schwarzberggletscher': {'temp': sion_dev_temp, 'prec': sion_dev_prec},
    'Hohlaubgletscher': {'temp': sion_dev_temp, 'prec': sion_dev_prec},
    'Glacier du Giétro': {'temp': sion_dev_temp, 'prec': sion_dev_prec},
    'Silvrettagletscher': {'temp': davos_dev_temp, 'prec': davos_dev_prec},
    'Claridenfirn': {'temp': altdorf_dev_temp, 'prec': altdorf_dev_prec}
}

# Selected columns for analysis
temp_cols = ['may_td', 'june_td', 'july_td', 'august_td', 'september_td']
prec_cols = ['october_pd', 'november_pd', 'december_pd', 'january_pd', 'february_pd', 'march_pd', 'april_pd']

# Process each glacier
for glacier_name, weather_data in glacier_mappings.items():
    print(f"\n{'='*80}")
    print(f"Regression Analysis for {glacier_name}")
    print('='*80)



    try:
        # Extract mass balance data
        mb_data = mass_balance_df[mass_balance_df['glacier name'] == glacier_name][['annual mass balance (mm w.e.)']].copy()
        mb_data = mb_data.reset_index(drop=True)

        if len(mb_data) == 0:
            print(f"No mass balance data found for {glacier_name}")
            continue

        # Special handling for Claridenfirn
        if glacier_name == 'Claridenfirn':
            # Create copies of the weather dataframes
            temp_df = weather_data['temp'][temp_cols].copy()
            prec_df = weather_data['prec'][prec_cols].copy()

            # Get the hydrological year for each row
            temp_years = weather_data['temp']['hydrological year']
            prec_years = weather_data['prec']['hydrological year']

            # Remove rows where hydrological_year is '1993-1994' or '1994-1995'
            temp_df = temp_df[~temp_years.isin(['1993-1994', '1994-1995'])].reset_index(drop=True)
            prec_df = prec_df[~prec_years.isin(['1993-1994', '1994-1995'])].reset_index(drop=True)
        else:
            # For other glaciers, use all weather data
            temp_df = weather_data['temp'][temp_cols].reset_index(drop=True)
            prec_df = weather_data['prec'][prec_cols].reset_index(drop=True)
            
        # Make sure we don't use more weather data than mass balance data
        max_index = min(len(mb_data) - 1, len(temp_df) - 1, len(prec_df) - 1)
        temp_df = temp_df.iloc[0:max_index+1]
        prec_df = prec_df.iloc[0:max_index+1]

        # Merge data
        reg_data = mb_data.iloc[0:max_index+1].copy()
        reg_data = pd.concat([reg_data, temp_df, prec_df], axis=1)

        # Prepare for regression
        X = reg_data[temp_cols + prec_cols]
        y = reg_data['annual mass balance (mm w.e.)']

        # Add constant and drop NA
        X = sm.add_constant(X)
        reg_data_clean = pd.concat([X, y], axis=1).dropna()

        if len(reg_data_clean) == 0:
            print(f"No valid data remaining for {glacier_name} after cleaning")
            continue

        # Perform regression
        model = sm.OLS(reg_data_clean['annual mass balance (mm w.e.)'],
                      reg_data_clean.drop('annual mass balance (mm w.e.)', axis=1)).fit()

        # Print results
        print(f"\nNumber of observations: {len(reg_data_clean)}")
        print("\nRegression Summary:")
        print(model.summary())

        print("\nCoefficient Interpretation:")
        for param in model.params.index:
            if param == 'const':
                print(f"Intercept (normal mass balance): {model.params[param]:.2f} (p={model.pvalues[param]:.4f})")
            else:
                print(f"{param}: {model.params[param]:.2f} (p={model.pvalues[param]:.4f})")

        # Check multicollinearity
        print("\nVariance Inflation Factors (VIF):")
        vif_data = pd.DataFrame()
        vif_data["Variable"] = X.columns
        vif_data["VIF"] = [variance_inflation_factor(reg_data_clean.drop('annual mass balance (mm w.e.)', axis=1).values, i)
                          for i in range(len(X.columns))]
        print(vif_data)

    except Exception as e:
        print(f"Error processing {glacier_name}: {str(e)}")



Regression Analysis for Grosser Aletschgletscher

Number of observations: 111

Regression Summary:
                                  OLS Regression Results                                 
Dep. Variable:     annual mass balance (mm w.e.)   R-squared:                       0.760
Model:                                       OLS   Adj. R-squared:                  0.731
Method:                            Least Squares   F-statistic:                     25.86
Date:                           Wed, 03 Dec 2025   Prob (F-statistic):           3.55e-25
Time:                                   13:28:40   Log-Likelihood:                -810.49
No. Observations:                            111   AIC:                             1647.
Df Residuals:                                 98   BIC:                             1682.
Df Model:                                     12                                         
Covariance Type:                       nonrobust                                         
